# Modelisation de la segregation

inspiration: https://ncase.me/polygons/ 

Le but de ce TD est de refaire cette modélisation de la ségrégation.
Si vous le voulez, vous pouvez essayer de le faire par vous même. Sinon, ce notebook est là pour vous aider, étape par étape.

Pour chaque fonction que vous codez, essayez d'évaluer la complexité de votre implémentation.

In [ ]:
%matplotlib inline
# explication de cette ligne en fin de TD

## 1. Modélisation du monde dans l'état initial

Premièrement, il faut construire le monde dans lequel évolueront les populations. Pour cela, nous allons créer une classe `World`. Le monde sera représenté par un numpy array de 2 dimension (= une matrice), stocké dans l'attribut `grid` de cette classe. Pour l'instant, écrivez cette classe avec uniquement le constructeur. Ce dernier prendra en argument la hauteur de la grille `height`, créera la grille de la bonne hauteur et la stockera dans l'attribut `grid`. Cette grille sera initialisé avec des 0.

**Indice**: regarder la fonction zeros de numpy. Attention, elle renvoit des `float`, donc vous devrez convertir le résultat en `int` avec la méthode astype des `arrays`.

**Convention**: on abrège numpy par np. Vous pouvez mettre ceci dans le code avec la ligne de code `import numpy as np`. Ainsi, vous devrez écrire `np.array` au lieu de `numpy.array`.

In [ ]:
# Votre classe ici

In [ ]:
# Test du constructeur
test = World(height=10)
if test.grid.shape != (10, 10):
    print("Votre grille n'est pas de la bonne taille.")
if not(np.all(test.grid == 0)):
    print("Votre grille ne contient pas que des zéros.")

Nous allons chercher à remplir aléatoirement la grille avec des habitants. Pour cela, commencons par écrire une fonction `fill_randomly` prenant en argument un array `arr`, un élément `e` et un nombre `n`, et qui insère dans `arr` `n` fois l'élément `e` à des positions aléatoires.

Attention, la fonction ne doit rien retourner, mais dois agir directement sur `arr`. De plus, elle ne doit remplacer que des 0. Si il n'y a pas assez de 0 dans `arr`, elle renvoit une erreur `ValueError('Not enough place')`. 

Vous pourrez utiliser les fonctions contenu dans `numpy.random`.

**Indice 1**: commencez par extraire la position de tous les 0 dans `arr`.

**Indice 2**: une fois que avez testé si il y a assez de place, vous pouvez utiliser la fonction `shuffle` de `numpy.random` pour mélanger les positions, puis choisir les `n` premières pour mettre `elem`.

In [ ]:
# Votre fonction fill_randomly

In [ ]:
# Test de la fonction fill_randomly
arr = np.zeros((10, 10))
fill_randomly(arr, 1, 5)
if np.sum(arr == 1) != 5:
    print('Vous avez une erreur')
fill_randomly(arr, 2, 95)
if np.sum(arr == 0):
    print('Vous avez remplacer un 1')
try:
    fill_randomly(arr, 3, 1)
    print("Vous ne renvoyer pas d'erreur quand il n'y a plus de place")
except:
    pass

Nous allons maintenant intégrer dans notre classe `World` une fonction pour remplir le monde d'habitants. Pour cela, écrivez une méthode `fill_world` qui prendra en argument le nombre de populations différentes `n_pop`, ainsi que pour chaque population le pourcentage de la grille à remplir. Ceci sera modélisé par une liste `percent_pop` de taille `n_pop` dont la somme devra être inférieur à 1.

Par exemple, `test.fill_world(2, [0.1, 0.5])` remplira la `grid` de test avec 10% de 1 et 50% de 2.  
Attention: si on appelle ensuite `test.fill_world(1, [0.2])`, la `grid` de test contiendra 20% de 3!

**Indice**: vous allez devoir sauvegarder le dernier nombre utilisé pour représenter une population. Pour ce faire, modifier le constructeur !

Vous pouvez copier coller le code de la classe World dans la cellule suivante.

In [ ]:
# Votre classe ici

In [ ]:
# Test de la méthode fill_world
test = World(10)
test.fill_world(3, [0.1, 0.1, 0.1])
for i in range(1, 4):
    if np.sum(test.grid == i) != 10:
        print('Vous avez une erreur !')
test.fill_world(1, [0.5])
if np.sum(test.grid == 4) != 50:
    print('Vous avez une erreur quand on appelle plusieurs fois la fonction !')
try:
    test.fill_world(1, [0.5])
    print('Vous ne gérez pas bien le manque de place !')
except:
    pass

Pour finir, créer une fonction create_world, qui prendra en argument la taille de la grid `height`, le nombre de population `n_pop`, et `percent_pop` définit comme dans fill_world, et qui renverra le monde créé. `n_pop` aura une valeur par défaut de `2`, et `percent_pop` aura une valeur par défaut de `[0.4, 0.4]`. 

In [ ]:
# fonction create_world

In [ ]:
# Test de create_world

world = create_world(10)
if np.sum(world.grid == 1) == 40 and np.sum(world.grid == 2) == 40:
    print('Fin de la partie 1')
else:
    print('Vous avez une erreur')

## 2. Trouvons qui est malheureux

La prochaine étape est de savoir qui veux déménager. Pour cela, il faut déjà définir une condition. Pour l'instant, cette condition sera fixé et vous pourrez la coder en dur (pas besoin de mettre de paramètre en argument). Disons que quelqu'un veut déménager si strictement plus de 50% de ses voisins sont différents.

Les cases du centre ont 8 voisins: sur les côtés et sur les diagonales!

Commencons par écrire une fonction `want_to_move` qui, à partir d'une grille et d'une position, renvoit `True` si la personne veut déménager et `False` sinon. La grille est un numpy array, la position est un 2-uple.

**Note**: si une personne n'a pas de voisins, elle ne veut pas déménager. Si on appelle cette fonction sur une case vide, elle renvoit `False`.

In [ ]:
# want_to_move

In [ ]:
# Test de want_to_move
test = create_world(10)
test.grid = np.load('./test_world.npy')
print(test.grid)
sol = np.load('wants_to_move.npy')
pred = np.array([want_to_move(test.grid, (x, y)) for x in range(10) for y in range(10)])
print(f' Vous avez raison à {np.mean(sol == pred)*100}%')

Maintenant que nous avons cette fonction, écrivons une fonction qui renvoie une personne aléatoire qui à envie de déménager ainsi que la position d'un espace libre. Cette fonction sera nommé `next_move`, et prendra en argument un numpy array `grid`.

Nous supposons dans cette fonction que quelqu'un veut déménager et qu'il y a au moins une place de libre.

**Indice**: aidez vous de la fonction `fill_randomly`.

In [ ]:
# fonction next_move

In [ ]:
# Test de next_move
move_from, move_to = next_move(test.grid)
if not(want_to_move(test.grid, move_from)) or test.grid[move_to] != 0:
    print('Vous avez une erreur')

## 3. Finalisons le programme

Il nous manque que très peu de chose pour avoir un programme qui fonctionne. Il nous manque une fonction pour savoir si la grille est finie(=plus personne ne veut bouger), une fonction pour effectuer un déplacement et une fonction pour boucler tant qu'elle ne l'est pas.

Commencons par le déplacement. Cette fonction sera la méthode `move` de World. Vous pouvez donc refaire un copier coller. Elle prendra en argument une position initiale, une position finale, et intervertira ces deux positions. 

In [ ]:
# Votre classe ici


In [ ]:
# Test de want_to_move
test = create_world(10)
test.grid = np.load('./test_world.npy')
test.move((0, 0), (9, 8))
if test.grid[0, 0] != 0 or test.grid[9, 8] != 2:
    print('Vous avez une erreur')

Ecrivons les deux dernières fonctions. La première sera `is_complete` et prendra en argument un numpy array `grid`. La seconde sera `run_simulation` et prendra comme argument une instance de la classe `World`, ainsi qu'un nombre maximal d'itération `iter_max`. Celui-ci aura par défaut une valeur de 100, et aura pour rôle d'empécher que la fonction tourne indéfiniment. Il définit en effet le nombre maximal de déplacement que l'on peut faire.

`run_simulation` affichera le numéro de l'itération en cours à chaque début de boucle, ainsi que le temps pris pour faire l'itération à la fin de chaque boucle. Pour cela, vous aurez besoin du package `time`.  
La fonction renverra `True` si le monde est stable, `False` sinon.

In [ ]:
# vos fonctions is_complete et run_simulation

In [ ]:
test = create_world(10)
test.grid = np.load('./test_world.npy')
run_simulation(test)
print(test.grid)

## 4. Analyse des résultats

Maintenant que nous avons un programme fonctionnel, examinons les résultats. Cela peut se faire à deux niveaux.  
Tout d'abord, restons dans notre problème et examinons la ségrégation dans ce monde. Pour ce faire, nous pouvons utiliser plusieurs métriques. Par exemple, nous pouvons regarder quel est le pourcentage d'habitants ayant au moins un voisin différent. Nous pourrions aussi décider de regarder le pourcentage moyen de voisins différents. 

Implémenter une de ces métriques, d'abord pour un individu dans une grille (fonction ayant pour argument `grid` et une position `pos`), puis dans la classe `World`, travaillant sur toute la grille et donnant la métrique moyenne. N'oubliez pas de tester vos fonctions.

In [ ]:
# La métrique


In [ ]:
# Test de la métrique
test = create_world(10)
test.grid = np.load('./test_world.npy')
test.move((4, 6), (4, 7))
test.move((5, 6), (6, 7))
print(test.grid)
print(has_a_different_neighbor(test.grid, (0, 0)))
print(has_a_different_neighbor(test.grid, (9, 9)))
print(has_a_different_neighbor(test.grid, (4, 5)))
print(has_a_different_neighbor(test.grid, (4, 6)))

In [ ]:
# La classe World


In [ ]:
# Test de la classe World
test = create_world(10)
print(f'segregation: {test.compute_segregation()}')
run_simulation(test)
print(test.grid)
print(f'segregation: {test.compute_segregation()}')

Vous pouvez maintenant changer la fonction `run_simulation` pour qu'elle renvoit l'évolution de la ségrégation au cours du temps. Pour ce faire, calculer celle-ci à chaque itération, et renvoyer une liste avec ces résultats, en plus de `True` ou `False`.

In [ ]:
# fonction run_simulation

In [ ]:
# Test de la classe World
test = create_world(10)
res, segregs = run_simulation(test)
print(test.grid)
print(segregs)

Pour finir, sur cette première analyse, tracer l'évolution de la ségrégation grâce à `matplotlib.pyplot`. Par convention, importez le et renommez le en `plt`. Vous pourrez alors utiliser les fonctions `plot` et `show`.

**Note**: la première ligne de code de ce notebook `%matplotlib inline`, permet d'afficher les graphiques dans les notebooks.

In [ ]:
# tracage avec plt

Passons à la deuxième étape de l'analyse. Analysons les performances du programme en fonction de la taille de l'input. Pour cela, dans la fonction `run_simulation`, renvoyer le temps moyen d'une itération.

In [ ]:
# Fonction run_simulation

In [ ]:
test = create_world(10)
res, segregs, mean_time = run_simulation(test)
print(test.grid)
print(segregs)
print(mean_time)

Enfin, écrivez une fonction `evaluate_perf`, qui prend en entré une liste de taille de grille, qui compile le temps moyen d'une itération pour chaque grille, et qui trace ce temps moyen en fonction de la taille de la grille.

In [ ]:
# fonction evaluate_perf

In [ ]:
# Attention, c'est long !
evaluate_perf([5, 10, 25, 50, 100, 200, 300, 400, 500])

## Pour aller plus loin

Vous pouvez ajouter ce que vous voulez à ce programme. Néanmoins, voici quelques pistes:
- mettre un argument `verbose` dans `run_simulation` pour controler l'affichage des itérations/temps par itération
- passer le tout en fichier et faire une meilleure architecture
- passer le seuil de mécontentement en paramètre pour pouvoir le faire varier
- ajouter un seuil de mécontentement de l'autre côté du spectre: quelqu'un démanage si moins de X% de ses voisins sont différents
- créer un système pour suivre l'évolution du monde au fur et à mesure de la simulation

Enfin, il y a une grosse amélioration à faire en terme de performance. Pour l'instant, on cherche sur toute la grille si quelqu'un veut déménager, les espaces libres, ... Vous pouvez améliorer ceci en conservant dans une liste ou une file les gens malheureux ainsi que les espaces libres, et en mettant à jour cette liste à chaque étape. La complexité en sera fortement réduite!